In [2]:
import numpy as np

In [3]:
img = np.array([
    [2, 2, 6, 4, 8, 5, 4, 4,],
    [1, 1, 9, 3, 3, 5, 5, 7,],
    [0, 0, 0, 6, 0, 4, 9, 8,],
    [3, 3, 8, 5, 6, 7, 7, 5,],
    [4, 2, 5, 7, 1, 6, 0, 6,],
    [1, 2, 7, 0, 4, 0, 2, 1,],
    [8, 6, 7, 2, 6, 8, 1, 0,],
    [8, 9, 0, 1, 4, 2, 1, 2,],
])
column, row = img.shape

mask = np.array([
    [1, 1, 1],
    [1, 1, 1],
    [1, 1, 1],
])

mask = mask / np.sum(mask)
img_convolution = np.zeros((column - 2, row - 2))
print(img[0:3, 0:3])
print("{} , {}".format(column, row))
for index_column in range(0, column - 2):
    for index_row in range(0, column - 2):
        next_column = index_column + 3
        next_row = index_row + 3
        sub_img = img[index_column:next_column, index_row:next_row]
        sub_img = (sub_img * mask) / np.max(img)
        img_convolution[index_column, index_row] = sub_img.sum()
print("New image with convolution")
print(img_convolution)

[[2 2 6]
 [1 1 9]
 [0 0 0]]
8 , 8
New image with convolution
[[0.25925926 0.38271605 0.48148148 0.4691358  0.5308642  0.62962963]
 [0.30864198 0.43209877 0.49382716 0.48148148 0.56790123 0.7037037 ]
 [0.30864198 0.44444444 0.4691358  0.51851852 0.49382716 0.64197531]
 [0.43209877 0.48148148 0.5308642  0.44444444 0.40740741 0.41975309]
 [0.51851852 0.4691358  0.48148148 0.41975309 0.34567901 0.2962963 ]
 [0.59259259 0.41975309 0.38271605 0.33333333 0.34567901 0.20987654]]


In [53]:
X = img_convolution.ravel()
print(X.shape)

number_neural_network = 36
number_weight = 36
layer_1 = np.random.randn(number_neural_network, number_weight)
bias = np.ones(number_neural_network)
output = np.zeros(number_neural_network)
for index in range(0,number_neural_network):
    sum_weight = X * layer_1[index]
    output[index] = bias[index] + sum_weight.sum()
print(output)
activation=1/(1 + np.exp(-output))
print(activation)
activation_round = np.clip(np.round(activation),0,1)
print(activation_round)

(36,)
[-1.58127547  2.76388212 -8.98802488  4.48548693  0.17880713  0.65057227
  3.50530098  0.35115142  2.31811891 -1.94095709  1.31287298  3.43497009
  1.87048571 -0.6885349   0.11572658 -0.84092195 -0.76673699  0.67985364
 -2.61553495  2.77670101  0.02647481 -1.2305213  -3.75772572 -0.69813774
  8.25800973 -1.20715542  3.92092411 -3.0641435   1.74835095  3.19586876
  5.49567317  1.95624424 -0.94712691 -0.57375871  1.36955278 -2.56427937]
[1.70614920e-01 9.40692588e-01 1.24880939e-04 9.88854231e-01
 5.44583061e-01 6.57139410e-01 9.70838222e-01 5.86896767e-01
 9.10366563e-01 1.25542747e-01 7.87993512e-01 9.68779741e-01
 8.66514468e-01 3.34359070e-01 5.28899398e-01 3.01340645e-01
 3.17185383e-01 6.63706031e-01 6.81452851e-02 9.41403728e-01
 5.06618315e-01 2.26090199e-01 2.28045697e-02 3.32225244e-01
 9.99740893e-01 2.30204754e-01 9.80562536e-01 4.46107706e-02
 8.51744688e-01 9.60678515e-01 9.95912285e-01 8.76125918e-01
 2.79462991e-01 3.60369976e-01 7.97307889e-01 7.14730219e-02]
[0. 1